In [ ]:
!pip install accelerate
!pip install optimum
!pip install auto-gptq
!pip install transformers==4.37.2
!pip install langchain
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0").to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Load URLs
loaders = [
    WebBaseLoader("https://www.geeksforgeeks.org/introduction-to-linux-operating-system/"),
    WebBaseLoader("https://linuxsimply.com/linux-tutorial/")
]

docs = []
for loader in loaders:
  docs.extend(loader.load())

print("Number of documents:{}".format(len(docs)))
print("Page size of document:{}".format(len(docs[0].page_content)))
#print("Sample Document:{}".format(docs[0].page_content[5000:5500]))

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1500,
      chunk_overlap = 150
  )

splits = text_splitter.split_documents(docs)
print("Total Number of Splits:{}".format(len(splits)))


persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

Number of documents:2
Page size of document:31505
Sample Document:nterview QuestionsAndroidAndroid TutorialAndroid Studio TutorialKotlin For AndroidAndroid ProjectsAndroid Interview Questions6 Weeks of Android App DevelopmentExcelMS Excel TutorialIntroduction to MS ExcelData Analysis in ExcelBasic Excel Formulas & FunctionsData Analysis in Advanced ExcelWorkbooksStatistical FunctionsData Visualization in ExcelPivot Tables in ExcelExcel Spreadsheets in PythonBasic Excel ShortcutsMathematicsNumber SystemAlgebraLinear AlgebraTrigonometrySet TheoryStatisticsProbab
Total Number of Splits:37


In [ ]:
def create_prompt_template(prompt):
  context = vectordb.similarity_search(prompt,k=3)[0].page_content
  prompt_template=f'''[INST] <<SYS>>
  Answer the following QUESTION based on the CONTEXT given or on the previous QUESTIONS or ANSWERS. If you do not know the answer and the CONTEXT doesn't contain the answer truthfully say "I don't know".<</SYS>>
  CONTEXT: {context}

  QUESTION: {prompt}[/INST]


  '''
  print(context)
  print('end of context')
  return prompt_template

In [ ]:
prompt = "What are the fundamental commands of file system navigation in Linux?"
prompt_template = create_prompt_template(prompt)
encoded_input = tokenizer(prompt_template, return_tensors='pt').to('cuda')
output = model.generate(**encoded_input).to('cuda')
print(tokenizer.decode(output[0]))
#print(pipe(prompt_template)[0]['generated_text'])

<s> [INST] <<SYS>>
  Answer the following QUESTION based on the CONTEXT given. If you do not know the answer and the CONTEXT doesn't contain the answer truthfully say "I don't know".<</SYS>>
  CONTEXT: Linux Basics
The Linux Basics section covers essential concepts for beginners. We explain what Linux distributions are and how they differ from one another, emphasizing popular distributions like Ubuntu, Fedora, and CentOS. We introduce the Linux File System Hierarchy, which organizes files and directories, and we compare the Command-Line Interface (CLI) and Graphical User Interface (GUI). Additionally, we discuss terminal emulators and shells, exploring options like Bash and Zsh.
Navigating the File System
Here, we dive into file system navigation in Linux. We introduce fundamental commands such as ls, cd, pwd, and mkdir, which allow users to list files, change directories, print the current directory, and create new directories. We explain how to work with directories and files, includ

In [ ]:
prompt = "What are the fundamental commands of file system navigation in Linux?"
prompt_template = create_prompt_template(prompt)
model_inputs = tokenizer([prompt_template], return_tensors="pt").to("cuda")
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(**model_inputs, do_sample=True, max_new_tokens=128).to('cuda')
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

ANSWER:
  The fundamental commands of file system navigation in Linux are ls, cd, pwd, and mkdir. These commands allow users to list files, change directories, print the current directory, and create new directories.


In [ ]:
messages = []
messages.append({"role": "user", "content": "What are the fundamental commands of file system navigation in Linux?"})
messages.append({"role": "system", "content": "The fundamental commands of file system navigation in Linux are ls, cd, pwd, and mkdir. These commands allow users to list files, change directories, print the current directory, and create new directories."})

In [ ]:
messages

[{'role': 'user',
  'content': 'What are the fundamental commands of file system navigation in Linux?'},
 {'role': 'system',
  'content': 'The fundamental commands of file system navigation in Linux are ls, cd, pwd, and mkdir. These commands allow users to list files, change directories, print the current directory, and create new directories.'}]

In [ ]:
promt = "Tell about the first fundamental commands"

messages.append({"role": "user", "content": create_prompt_template(prompt)})
model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=128).to('cuda')
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

The fundamental commands of file system navigation in Linux are:

1. ls - lists the files and directories in a particular directory or all directories beneath it. 
2. cd - changes directory to a new location; for local working directory, cd /my/directory, while for remote directories like FTP server, cd /username/folder/named/folder. 
3. pwd - shows current working directory. 
4. mkdir - creates a directory. 
5. chmod - changes the permissions of a file or directory. 
6. touch - creates a 'new' file or updates an


In [ ]:
promt = "What do the first fundamental commands of file system navigation in Linux"
messages.append({"role": "user", "content": prompt})
model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=128).to('cuda')
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

Sure! Here are the fundamental commands of file system navigation in Linux:

1. `ls`: This command is used to list files and directories within a directory.

2. `cd`: This command is used to change the directory you are currently working in.

3. `pwd`: This command is used to print the current working directory.

4. `mkdir`: This command is used to create new directories.

5. `rm`: This command is used to remove files from a directory or a specific file from a directory.

6. `cp`: This command is used


In [ ]:
messages = []
messages.append({"role": "user", "content": "What are the fundamental commands of file system navigation in Linux?"})
messages.append({"role": "system", "content": "The first and second fundamental commands of file system navigation in Linux are cd, mkdir>"})
promt = "What do the second fundamental commands of file system navigation in Linux"
messages.append({"role": "user", "content": create_prompt_template(prompt)})
model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=128).to('cuda')
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

The basic, essential functions of file system navigation in Linux include the commands of listing files using the "ls" command, changing directories via the "cd" command, print the current working directory using the "pwd" command, and creating and deleting files using the "mkdir" and "rm" commands. Other essential commands include file permissions and ownership, modifying the permissions and ownership using the "chmod" and "chown" commands, respectively.


In [ ]:
persist_directory = '/content/drive/MyDrive/Colab Notebooks/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
prompt = "What are the fundamental commands of file system navigation in Linux?"
prompt_template = create_prompt_template(prompt)
model_inputs = tokenizer([prompt_template], return_tensors="pt").to("cuda")
input_length = model_inputs.input_ids.shape[1]
generated_ids = model.generate(**model_inputs, do_sample=True, max_new_tokens=100).to('cuda')
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

Linux Basics
The Linux Basics section covers essential concepts for beginners. We explain what Linux distributions are and how they differ from one another, emphasizing popular distributions like Ubuntu, Fedora, and CentOS. We introduce the Linux File System Hierarchy, which organizes files and directories, and we compare the Command-Line Interface (CLI) and Graphical User Interface (GUI). Additionally, we discuss terminal emulators and shells, exploring options like Bash and Zsh.
Navigating the File System
Here, we dive into file system navigation in Linux. We introduce fundamental commands such as ls, cd, pwd, and mkdir, which allow users to list files, change directories, print the current directory, and create new directories. We explain how to work with directories and files, including copying, moving, and deleting them. Moreover, we cover file permissions and ownership, demonstrating how to set and modify permissions using chmod and how to change ownership with chown.
Essential L

In [ ]:
def get_answer(promt):
  prompt_template = create_prompt_template(prompt)
  model_inputs = tokenizer([prompt_template], return_tensors="pt").to("cuda")
  input_length = model_inputs.input_ids.shape[1]
  generated_ids = model.generate(**model_inputs, do_sample=True, max_new_tokens=100).to('cuda')
  return tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]

In [ ]:
prompt = "What are the Typical services include?"
get_answer(prompt)

1.1. IntroductionAll operating systems provide services for programs they run. Typical services include executing anew program, opening a file, reading a file, allocating a region of memory, getting the current time ofday, and so on. The focus of this text is to describe the services provided by various versions of theUNIX operating system.Describing the UNIX System in a strictly linear fashion, without any forward references to terms thathaven't been described yet, is nearly impossible (and would probably be boring). This chapterprovides a whirlwind tour of the UNIX System from a programmer's perspective. We'll give somebrief descriptions and examples of terms and concepts that appear throughout the text. We describethese features in much more detail in later chapters. This chapter also provides an introduction andoverview of the services provided by the UNIX System, for programmers new to this environment.

Answer: Typical services provided by the UNIX System include: 1. Execute anew

In [ ]:
prompt = "What is the popular Linux distributions?"
get_answer(prompt)

What is Linux
Linux is a powerful and flexible family of operating systems that are free to use and share. It was created by a person named Linus Torvalds in 1991. What’s cool is that anyone can see how the system works because its source code is open for everyone to explore and modify. This openness encourages people from all over the world to work together and make Linux better and better. Since its beginning, Linux has grown into a stable and safe system used in many different things, like computers, smartphones, and big supercomputers. It’s known for being efficient, meaning it can do a lot of tasks quickly, and it’s also cost-effective, which means it doesn’t cost a lot to use. Lots of people love Linux, and they’re part of a big community where they share ideas and help each other out. As technology keeps moving forward, Linux will keep evolving and staying important in the world of computers.
Linux Distribution
Linux distribution is an operating system that is made up of a colle

In [ ]:
prompt = "How to list all files in directory in Linux?"
get_answer(prompt)

Linux Basics
The Linux Basics section covers essential concepts for beginners. We explain what Linux distributions are and how they differ from one another, emphasizing popular distributions like Ubuntu, Fedora, and CentOS. We introduce the Linux File System Hierarchy, which organizes files and directories, and we compare the Command-Line Interface (CLI) and Graphical User Interface (GUI). Additionally, we discuss terminal emulators and shells, exploring options like Bash and Zsh.
Navigating the File System
Here, we dive into file system navigation in Linux. We introduce fundamental commands such as ls, cd, pwd, and mkdir, which allow users to list files, change directories, print the current directory, and create new directories. We explain how to work with directories and files, including copying, moving, and deleting them. Moreover, we cover file permissions and ownership, demonstrating how to set and modify permissions using chmod and how to change ownership with chown.
Essential L

In [ ]:
prompt = "Is There Any Difference between Linux and Ubuntu?"
print(get_answer(prompt))

Disadvantages of Linux

It is not very user-friendly. So, it may be confusing for beginners.
It has small peripheral hardware drivers as compared to windows.

Frequently Asked Questions in Linux Operating System
What is Linux Operating System?

Linux is an open-source operating system developed by Linus Torvalds in 1991. It provides a customizable and secure alternative to proprietary systems. With its stable performance, Linux is widely used across devices, from personal computers to servers and smartphones. The collaborative efforts of its developer community continue to drive innovation, making Linux a dominant force in the world of computing.

Is There Any Difference between Linux and Ubuntu? 

The answer is YES. The main difference between Linux and Ubuntu is Linux is the family of open-source operating systems which is based on Linux kernel, whereas Ubuntu is a free open-source operating system and the Linux distribution which is based on Debian. Or in other words, Linux is the c

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/linux.pdf")
pages = loader.load_and_split()

In [ ]:
pages[228]

Document(page_content='4.14Some versions of the finger(1) command output "New mail received ..." and "unreadsince ..." where ... are the corresponding times and dates. How can the programdetermine these two times and dates?4.15Examine the archive formats by the cpio(1) and tar(1) commands. (These descriptionsare usually found in Section 5 of the UNIX Programmer\'s Manual.) How many of thethree possible time values are saved for each file? When a file is restored, what value doyou think the access time is set to, and why?4.16Does the UNIX System have a fundamental limitation on the depth of a directory tree?To find out, write a program that creates a directory and then changes to that directory,in a loop. Make certain that the length of the absolute pathname of the leaf of thisdirectory is greater than your system\'s PATH_MAX limit. Can you call getcwd to fetch thedirectory\'s pathname? How do the standard UNIX System tools deal with this longpathname? Can you archive the directory usin

In [ ]:
docs_to_append = [pages[i] for i in range(38, 229)]

In [ ]:
len(docs_to_append)

191

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1500,
      chunk_overlap = 150
  )
splits = text_splitter.split_documents(docs_to_append)
len(splits)

304

In [ ]:
vectordb.add_documents(documents=splits)

In [ ]:
prompt = "What are the fundamental commands of file system in Linux?"
print(get_answer(prompt))

Linux Basics
The Linux Basics section covers essential concepts for beginners. We explain what Linux distributions are and how they differ from one another, emphasizing popular distributions like Ubuntu, Fedora, and CentOS. We introduce the Linux File System Hierarchy, which organizes files and directories, and we compare the Command-Line Interface (CLI) and Graphical User Interface (GUI). Additionally, we discuss terminal emulators and shells, exploring options like Bash and Zsh.
Navigating the File System
Here, we dive into file system navigation in Linux. We introduce fundamental commands such as ls, cd, pwd, and mkdir, which allow users to list files, change directories, print the current directory, and create new directories. We explain how to work with directories and files, including copying, moving, and deleting them. Moreover, we cover file permissions and ownership, demonstrating how to set and modify permissions using chmod and how to change ownership with chown.
Essential L